In [3]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# Paths to the directories for Set A and Set E data (update with your actual paths)
set_a_path = r"C:\Users\tanis\OneDrive\Desktop\audit_EEG\Z"  # Path to Set A (Normal)
set_e_path = r"C:\Users\tanis\OneDrive\Desktop\audit_EEG\S"  # Path to Set E (Epileptic Seizure)

# Load Set A (normal) data
set_a_data = []
for file_name in os.listdir(set_a_path):
    file_path = os.path.join(set_a_path, file_name)
    data = np.loadtxt(file_path)  # Load each file as an array of 4097 samples
    set_a_data.append(data)

# Load Set E (epileptic seizure) data
set_e_data = []
for file_name in os.listdir(set_e_path):
    file_path = os.path.join(set_e_path, file_name)
    data = np.loadtxt(file_path)  # Load each file as an array of 4097 samples
    set_e_data.append(data)

# Combine the datasets
X = np.array(set_a_data + set_e_data)  # Shape: (200, 4097) if each set has 100 segments
y = np.hstack((np.zeros(len(set_a_data)), np.ones(len(set_e_data))))  # Labels: 0 for normal, 1 for seizure

# Normalize the data to have zero mean and unit variance
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


In [4]:
import pywt

# Function to apply DWT and extract wavelet coefficients
def extract_wavelet_coefficients(signal):
    # Decompose signal with 'db4' wavelet at level 5
    coeffs = pywt.wavedec(signal, wavelet='db4', level=5)
    return coeffs

# Function to calculate statistical features from wavelet coefficients
def extract_wavelet_features(coeffs):
    features = []
    A5, D5, D4, D3, D2, D1 = coeffs

    # 1. Mean of absolute values (captures the average amplitude)
    features.append(np.mean(np.abs(A5)))
    features.append(np.mean(np.abs(D3)))
    features.append(np.mean(np.abs(D4)))
    features.append(np.mean(np.abs(D5)))

    # 2. Average power (energy of the signal within each sub-band)
    features.append(np.sum(np.square(A5)) / len(A5))
    features.append(np.sum(np.square(D3)) / len(D3))
    features.append(np.sum(np.square(D4)) / len(D4))
    features.append(np.sum(np.square(D5)) / len(D5))

    # 3. Standard deviation (measures variability in each sub-band)
    features.append(np.std(A5))
    features.append(np.std(D3))
    features.append(np.std(D4))
    features.append(np.std(D5))

    # 4. Ratio of absolute mean values of adjacent sub-bands (D3/D4 and D4/D5)
    features.append(np.mean(np.abs(D3)) / np.mean(np.abs(D4)))
    features.append(np.mean(np.abs(D4)) / np.mean(np.abs(D5)))

    return np.array(features)

# Apply DWT to each EEG segment and extract features
coefficients = [extract_wavelet_coefficients(segment) for segment in X_normalized]
X_features = np.array([extract_wavelet_features(coeff) for coeff in coefficients])

print("Feature Matrix Shape:", X_features.shape)  # Expected shape: (200, 14) if there are 14 features


Feature Matrix Shape: (200, 14)


In [5]:
from sklearn.decomposition import PCA

# Reduce dimensions to 10 components with PCA
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_features)

print("PCA Reduced Shape:", X_pca.shape)  # Shape should be (200, 10)


PCA Reduced Shape: (200, 10)


In [6]:
from sklearn.decomposition import FastICA

# Reduce dimensions to 10 components with ICA
ica = FastICA(n_components=10, random_state=42)
X_ica = ica.fit_transform(X_features)

print("ICA Reduced Shape:", X_ica.shape)  # Shape should be (200, 10)


ICA Reduced Shape: (200, 10)


In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Reduce dimensions with LDA (LDA reduces to n_classes - 1, here 1 component since binary classification)
lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_features, y)

print("LDA Reduced Shape:", X_lda.shape)  # Shape should be (200, 1)


LDA Reduced Shape: (200, 1)


In [50]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Split PCA features for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Initialize and train the SVM
svm = SVC(kernel='rbf', gamma='auto')
svm.fit(X_train, y_train)

# Step 6: Evaluation (Confusion Matrix, Sensitivity, Specificity)
y_pred = svm.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Sensitivity (True Positive Rate)
sensitivity = tp / (tp + fn)

# Specificity (True Negative Rate)
specificity = tn / (tn + fp)

accuracy = accuracy_score(y_test, y_pred)


# Print the results
print(f"Confusion Matrix:\n{cm}")
print(f"Accuracy: {accuracy * 100:.3f}")
print(f"Sensitivity: {sensitivity* 100:.3f}")
print(f"Specificity: {specificity* 100:.3f}")


Confusion Matrix:
[[31  0]
 [ 1 28]]
Accuracy: 98.333
Sensitivity: 96.552
Specificity: 100.000


In [49]:
# Split ICA features for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_ica, y, test_size=0.3, random_state=42)

# Train SVM
svm.fit(X_train, y_train)

# Step 6: Evaluation (Confusion Matrix, Sensitivity, Specificity)
y_pred = svm.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Sensitivity (True Positive Rate)
sensitivity = tp / (tp + fn)

# Specificity (True Negative Rate)
specificity = tn / (tn + fp)

accuracy = accuracy_score(y_test, y_pred)


# Print the results
print(f"Confusion Matrix:\n{cm}")
print(f'Accuracy: {accuracy * 100:.3f}')
print(f"Sensitivity: {sensitivity* 100:.3f}")
print(f"Specificity: {specificity* 100:.3f}")

Confusion Matrix:
[[31  0]
 [ 1 28]]
Accuracy: 98.333
Sensitivity: 96.552
Specificity: 100.000


In [48]:
# Split LDA features for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_lda, y, test_size=0.3, random_state=42)

# Train SVM
svm.fit(X_train, y_train)

# Step 6: Evaluation (Confusion Matrix, Sensitivity, Specificity)
y_pred = svm.predict(X_test)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Sensitivity (True Positive Rate)
sensitivity = tp / (tp + fn)

# Specificity (True Negative Rate)
specificity = tn / (tn + fp)

accuracy = accuracy_score(y_test, y_pred)


# Print the results
print(f"Confusion Matrix:\n{cm}")
print(f'Accuracy: {accuracy * 100:.3f}')
print(f"Sensitivity: {sensitivity* 100:.3f}")
print(f"Specificity: {specificity* 100:.3f}")

Confusion Matrix:
[[31  0]
 [ 0 29]]
Accuracy: 100.000
Sensitivity: 100.000
Specificity: 100.000
